In [2]:
import numpy as np
import pennylane as qml
from pennylane import numpy as npqml
import matplotlib.pyplot as plt

num_freqs = 5  # You can change the number of frequencies used in the quantum circuit
num_qubits = num_freqs + 1  # Number of qubits required for the quantum circuit

dev = qml.device('default.qubit', wires=num_qubits)
@qml.qnode(dev)
def circuit(params, x):
    # Quantum circuit to evaluate the function e^(-x)*sin(w*x) with different frequencies
    num_freqs = len(params) - 1  # The last parameter represents the constant term

    for i in range(num_freqs):
        qml.RX(params[i], wires=i)
        qml.RZ(x * params[i], wires=i)

    qml.RX(params[-1], wires=num_freqs)  # Last parameter is the constant term
    qml.strongly_entangling()
    return qml.expval(qml.PauliZ(0))

def cost(params, x,y):
    print('----------')
    print(qml.draw(circuit)(params, x))
    loss = 0
    for x in x:
      print(x)
      print(circuit(params, x))
    
    return circuit(params, x)

def fit_function_with_quantum(x, y, num_freqs, num_layers=1):


    # # Define the cost function and create the quantum node
    # cost_fn = lambda params: cost(params, x, y)
    # qnode = qml.QNode(cost_fn, dev)

    # Randomly initialize the variational parameters
    # params = np.random.randn(num_qubits * num_layers)
    params=npqml.random.randn(num_qubits * num_layers,requires_grad=True)
    # print(params)
    # Optimize the parameters using gradient descent
    opt = qml.GradientDescentOptimizer(stepsize=0.1)
    num_steps = 100
    for i in range(num_steps):
        # params = opt.step(cost, params,x,y)
        params,_cost=opt.step_and_cost(cost,params,x,y)
        print(params)
        print('iter: ',i,' cost: ',params,'n_x: ',x)


    return params

def plot_fit_results(x, y, params):
    y_pred = np.array([cost(params, xi) for xi in x])
    print(y_pred)
    # Plot the original data points and the fitted curve
    plt.figure(figsize=(8, 6))
    plt.scatter(x, y, label='Data Points', color='blue')
    plt.plot(x, y_pred, label='Fitted Curve', color='red', linewidth=2)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Function Fitting with Quantum Circuit')
    plt.legend()
    plt.show()

def main():
    # Input data points
    x = npqml.array([100.5, 1.5, 2.0, 3.0, 4.0, 5.0],requires_grad=False)
    y = npqml.array([0.2, 0.5, 0.7, 0.9, 1.0, 0.8],requires_grad=False)


    
    # Fit the function using a quantum circuit
    params = fit_function_with_quantum(x, y, num_freqs)
    
    # Plot the results
    plot_fit_results(x, y, params)

if __name__ == "__main__":
    main()

----------
0: ──RX(0.23)───RZ(M4)─┤  <Z>
1: ──RX(0.83)───RZ(M1)─┤     
2: ──RX(-0.83)──RZ(M0)─┤     
3: ──RX(0.45)───RZ(M3)─┤     
4: ──RX(1.01)───RZ(M2)─┤     
5: ──RX(-1.29)─────────┤     
100.5
Autograd ArrayBox with value 0.9747364255558473
1.5
Autograd ArrayBox with value 0.9747364255558476
2.0
Autograd ArrayBox with value 0.974736425555847
3.0
Autograd ArrayBox with value 0.9747364255558476
4.0
Autograd ArrayBox with value 0.9747364255558479
5.0
Autograd ArrayBox with value 0.9747364255558477
[tensor([ 0.24759422,  0.83357154, -0.83198745,  0.4469646 ,  1.00841024,
        -1.28726391], requires_grad=True), tensor([100.5,   1.5,   2. ,   3. ,   4. ,   5. ], requires_grad=False), tensor([0.2, 0.5, 0.7, 0.9, 1. , 0.8], requires_grad=False)]
iter:  0  cost:  [tensor([ 0.24759422,  0.83357154, -0.83198745,  0.4469646 ,  1.00841024,
        -1.28726391], requires_grad=True), tensor([100.5,   1.5,   2. ,   3. ,   4. ,   5. ], requires_grad=False), tensor([0.2, 0.5, 0.7, 0.9, 1. , 0.8],

ValueError: RX: wrong number(s) of dimensions in parameters. Parameters with ndims (2,) passed, (0,) expected.